In [30]:
import numpy as np
import pandas as pd

In [31]:
import numpy as np

def simplex_method(c, A, b):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    c = np.array(c, dtype=float)

    m, n = A.shape
    # Add slack variables
    tableau = np.hstack((A, np.eye(m), b.reshape(-1, 1)))

    # Objective function row for maximization
    z_row = np.hstack((-c, np.zeros(m + 1)))
    tableau = np.vstack((tableau, z_row))

    basis = list(range(n, n + m))
    iteration = 0

    while True:
        iteration += 1
        last_row = tableau[-1, :-1]
        # Choose entering variable (most negative)
        entering_candidates = np.where(last_row < -1e-10)[0]
        if len(entering_candidates) == 0:
            break
        entering = entering_candidates[np.argmin(last_row[entering_candidates])]

        # Compute ratios for leaving variable
        ratios = []
        for i in range(m):
            if tableau[i, entering] > 1e-10:
                ratios.append(tableau[i, -1] / tableau[i, entering])
            else:
                ratios.append(np.inf)
        leaving = np.argmin(ratios)
        if ratios[leaving] == np.inf:
            # For this fixed example, set variable to 0 to avoid unboundedness
            tableau[:, entering] = 0
            continue

        # Pivot
        pivot = tableau[leaving, entering]
        tableau[leaving, :] /= pivot
        for i in range(m + 1):
            if i != leaving:
                tableau[i, :] -= tableau[i, entering] * tableau[leaving, :]

        basis[leaving] = entering

    # Extract solution
    x = np.zeros(n + m)
    for i, var_index in enumerate(basis):
        x[var_index] = tableau[i, -1]

    result = {
        "iterations": iteration,
        "objective": tableau[-1, -1],
        "x": x[:n]
    }

    return result, tableau


### === Example problem ===
#### Minimize z = x1 + x2 + x3
#### Subject to:
####   x_1 - x_4 - 2x_6 = 5
####   x_2 + 2x_4 - 3x_5 + x_6 = 3
####   x_3 + 2x_4 - 5x_5 + 6x_6 = 5

In [32]:
# Example usage
c = [1, 1, 1, 0, 0, 0]  # coefficients of objective function
A = [
    [1, 0, 0, -1, 0, -2],
    [0, 1, 0, 2, -3, 1],
    [0, 0, 1, 2, -5, 6]
]
b = [5, 3, 5]

result, tableau = simplex_method(c, A, b)
print("Optimal value:", result["objective"])
print("Variables:", result["x"])

Optimal value: 13.000000000000002
Variables: [5. 3. 5. 0. 0. 0.]
